# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [1]:
# Download link 1
#!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

#!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

Tue May 17 09:46:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [2]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 599 kB/s            
     |████████████████████████████████| 3.3 MB 64.5 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.11.6
    Uninstalling tokenizers-0.11.6:
      Successfully uninstalled tokenizers-0.11.6
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2


## Import Packages

In [3]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(1314)

In [4]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 47 kB 378 kB/s            
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.5.1
    Uninstalling accelerate-0.5.1:
      Successfully uninstalled accelerate-0.5.1


## Load Model and Tokenizer




 

In [5]:
# model_name = "bert-base-chinese"
model_name = "luhua/chinese_pretrain_mrc_macbert_large"
#model = BertForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = BertTokenizerFast.from_pretrained(model_name)
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [6]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

#train_questions, train_paragraphs = read_data("../input/ml2022spring-hw7/hw7_train.json")
#dev_questions, dev_paragraphs = read_data("../input/ml2022spring-hw7/hw7_dev.json")
test_questions, test_paragraphs = read_data("../input/ml2022spring-hw7/hw7_test.json")

## Tokenize Data

In [7]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

#train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
#dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

#train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
#dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [8]:
train_batch_size = 8
doc_stride = 100

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs, doc_stride=doc_stride):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 40
        self.max_paragraph_len = 350
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = doc_stride

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            #mid = (answer_start_token + answer_end_token) // 2
            #paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            #paragraph_end = paragraph_start + self.max_paragraph_len
            
            start_min = max(0, answer_end_token - self.max_paragraph_len + 1)
            start_max = min(answer_start_token, len(tokenized_paragraph) - self.max_paragraph_len)
            start_max = max(start_min, start_max)
            paragraph_start = random.randint(start_min, start_max + 1)
            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask



#train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
#dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)


# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
#train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
#dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Function for Evaluation

In [9]:
def evaluate(data, output, doc_stride=doc_stride, token_type_ids=None, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        
        token_type_id = data[1][0][k].detach().cpu().numpy()
        #[CLS] + [question] + [SEP] + [paragraph] + [SEP]
        paragraph_start = token_type_id.argmax()
        paragraph_end = len(token_type_id) - 1 - token_type_id[::-1].argmax() - 1
        
        if start_index > end_index or start_index < paragraph_start or end_index > paragraph_end:
            continue
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        if prob > max_prob:
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
            # 找到tokenized paragraph中对应的位置
            origin_start = start_index + k * doc_stride - paragraph_start
            origin_end = end_index + k * doc_stride - paragraph_start;
        
    answer = answer.replace(' ', '')
    if '[UNK]' in answer:
        print('发现 [UNK]，这表明有文字无法编码, 使用原始文本')
        #print("Paragraph:", paragraph)
        #print("Paragraph:", paragraph_tokenized.tokens)
        print('--直接解码预测:', answer)
        #找到原始文本中对应的位置
        raw_start =  paragraph_tokenized.token_to_chars(origin_start)[0]
        raw_end = paragraph_tokenized.token_to_chars(origin_end)[1]
        answer = paragraph[raw_start:raw_end]
        print('--原始文本预测:',answer)
    
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer, max_prob

## Testing

In [10]:
print("Evaluating Test Set ...")

result = []
models_path = ['../input/hw7models/saved_model', '../input/hw7models/saved_model_1', '../input/hw7models/saved_model_2']
models = []
for path in models_path:
    model = BertForQuestionAnswering.from_pretrained(path).to(device)
    model.eval()
    models.append(model)

with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        answer, max_prob = '', 0
        for model in models:
            output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
            answer_i, prob_i = evaluate(data, output, doc_stride=doc_stride, paragraph=test_paragraphs[test_questions[i]["paragraph_id"]],
                               paragraph_tokenized=test_paragraphs_tokenized[test_questions[i]["paragraph_id"]])
            if prob_i > max_prob:
                answer = answer_i
        result.append(answer)

result_file = "result.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 大型購物中心[UNK]開幕
--原始文本预测: 大型購物中心MegaBox開幕
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 大型購物中心[UNK]開幕
--原始文本预测: 大型購物中心MegaBox開幕
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 大型購物中心[UNK]開幕
--原始文本预测: 大型購物中心MegaBox開幕
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 溥[UNK]
--原始文本预测: 溥儁
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 溥[UNK]
--原始文本预测: 溥儁
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 溥[UNK]
--原始文本预测: 溥儁
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 人均[UNK]都不如東南亞的大城市
--原始文本预测: 人均GDP都不如東南亞的大城市
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
--原始文本预测: 目前沒有觀察到任何語言純綷以力道來區分不同輔音
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
--原始文本预测: 目前沒有觀察到任何語言純綷以力道來區分不同輔音
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: 目前沒有觀察到任何語言純[UNK]以力道來區分不同輔音
--原始文本预测: 目前沒有觀察到任何語言純綷以力道來區分不同輔音
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]人國
--原始文本预测: 荇人國
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]人國
--原始文本预测: 荇人國
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--直接解码预测: [UNK]人國
--原始文本预测: 荇人國
发现 [UNK]，这表明有文字无法编码, 使用原始文本
--